In [20]:
import numpy as np
import pandas as pd
import matplotlib

import speech_recognition as sr
import soundfile
import spacy
import librosa
import glob
import os
import collections
from pydub import AudioSegment
from pydub.silence import split_on_silence
import pickle
import re
from pathlib import Path
from dataclasses import dataclass

from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, f1_score

In [21]:
@dataclass
class noun:
    item: str
    weight: float
    sentence: int
    precedence: int
    index: int
        
def populate_nouns(chunk_filename, text):
    tokens = nlp(text)
    idx = 1
    for token in tokens:
        if token.pos_ == 'NOUN':
            w = re.search(fr'\b({token.text})\b', text)
            sentence = chunk_filename.split(".")[0]
            curr_noun = noun(token.text, 100, sentence, idx, w.start())
            sentences.get(chunk_filename).append(curr_noun)
            idx += 1

In [22]:
file = Path.cwd() / 'fillers.txt'
with open(file, 'r') as text:
    data = text.read()
    data = re.sub(r"(\/\*.*?\*\/)", "", data, flags=re.M|re.DOTALL)
#print(data.strip())
fillers = data.split("\n")
fillers = [i for i in fillers if i != ''] 

In [23]:
audio = Path.cwd() / 'ted-audiofile.wav'
sentences = collections.defaultdict(dict)
nouns = collections.defaultdict(dict)

r = sr.Recognizer()
#chunks the current audio file, sr to translate audio chunk to text
def transcribe(path):
    sound = AudioSegment.from_wav(path)  
    chunk_list = split_on_silence(sound,min_silence_len = 900,silence_thresh = sound.dBFS-14,keep_silence=700,)
    folder_name = "audio-chunks"
    if not os.path.isdir(folder_name):
        os.mkdir(folder_name)
    whole_text = ""
    for i, audio_chunk in enumerate(chunk_list, start=1):
        chunk_filename = os.path.join(folder_name, f"chunk{i}.wav")
        name = f"chunk{i}.wav"
        sentences[name] = list()
        audio_chunk.export(chunk_filename, format="wav")
        with sr.AudioFile(chunk_filename) as source:
            audio_listened = r.record(source)
            try:
                text = r.recognize_google(audio_listened)
            except sr.UnknownValueError as e:
                print("Error:", str(e))
            else:
                text = f"{text.capitalize()}. "
                print(chunk_filename, ":", text)
                populate_nouns(name, text)
                whole_text += text
    return whole_text

In [24]:
#print("\nFull text:", transcribe(str(audio)))
nlp = spacy.load('en_core_web_sm')
full_text = transcribe(str(audio))

/Users/vpathalam/opt/anaconda3/lib/python3.7/site-packages/spacy/util.py:275: UserWarning: [W031] Model 'en_core_web_sm' (2.2.0) requires spaCy v2.2 and is incompatible with the current spaCy version (2.3.4). This may lead to unexpected results or runtime errors. To resolve this, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


Error: 
audio-chunks/chunk2.wav : Today i'm going to like talk about talking. 
audio-chunks/chunk3.wav : Last year in my elective class. 
audio-chunks/chunk4.wav : We would like told that we were going to have like this big linguistics project. 
audio-chunks/chunk5.wav : Since i didn't really know anything about linguistics apartments like how we speak. 
audio-chunks/chunk6.wav : My mom is like always going on and on about how i use like too much. 
audio-chunks/chunk7.wav : I thought that it would be like the perfect time to like look into it more. 
audio-chunks/chunk8.wav : My project was centered on looking at the use of like as a filler word. 
audio-chunks/chunk9.wav : In regards to gender age and education. 
audio-chunks/chunk10.wav : Is my results can help me argue that just as i have used like several times since starting my talk. 
audio-chunks/chunk11.wav : There's no link between an increased use of like and a lower level of education. 
audio-chunks/chunk12.wav : The word like 

In [25]:
def calculate_weight(text):
    for hesitation in fillers:
        for word in text.split(" "):
            if hesitation == word:
                print(hesitation)
                w = re.search(fr'\b({hesitation})\b', text)
                for key, value in sentences.items():
                    for i in range(len(value))[:-1]:
                        print(value[i])
                        between = w.start() < (value[i+1].index) and w.start() > (value[i].index)
                        first = w.start() < (value[i].index)
                        if i == 0:
                            if first:
                                value[i].weight = value[i].weight - 20
                                break
                        elif between:
                            value[i].weight = value[i].weight - 20
                            break
                
calculate_weight('I thought that it would be like the perfect time to like look into it more.')

like
noun(item='year', weight=100, sentence='chunk3', precedence=1, index=5)
noun(item='linguistics', weight=100, sentence='chunk4', precedence=1, index=60)
noun(item='linguistics', weight=100, sentence='chunk5', precedence=1, index=42)
noun(item='project', weight=100, sentence='chunk8', precedence=1, index=3)
noun(item='use', weight=100, sentence='chunk8', precedence=2, index=42)
noun(item='filler', weight=100, sentence='chunk8', precedence=3, index=59)
noun(item='regards', weight=100, sentence='chunk9', precedence=1, index=3)
noun(item='gender', weight=100, sentence='chunk9', precedence=2, index=14)
noun(item='age', weight=100, sentence='chunk9', precedence=3, index=21)
noun(item='results', weight=100, sentence='chunk10', precedence=1, index=6)
noun(item='times', weight=100, sentence='chunk10', precedence=2, index=70)
noun(item='link', weight=100, sentence='chunk11', precedence=1, index=11)
noun(item='use', weight=100, sentence='chunk11', precedence=2, index=37)
noun(item='level', we

In [15]:
#load RAVDESS dataset: https://zenodo.org/record/1188976

map<noun, weight]
map<chunk, arr[num hesitations, map<nouns, weight>]>


store  # currently a defaultdict type
defaultdict(<type 'dict'>, {'dict1': {'innerkey': 'value'}})
dict(d)  # but is exactly like a normal dictionary.
{'dict1': {'innerkey': 'value'}}

order in sentence
noun
which chunk
weight

noun(weight=0.1, sentence=1, precedence=2)
